In [ ]:
"""
File: emotion_recogniser_04.py
Author: Gabor Levai
Email: levaigabor.net@gmail.com
Description: Emotion recognition with CNN
"""
import pandas as pd
import numpy as np
import itertools
#matplotlib inline
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

from sklearn.metrics import confusion_matrix

#%% Data inspection

train = pd.read_csv("training.csv")
test = pd.read_csv("publictest.csv")
train.head()

print ("The number of traning set samples: {}".format(len(train)))
print ("The number of testing set samples: {}".format(len(test)))
#%%
print ("Training dataset emotion and its count")
train.emotion.value_counts()
#%%
print ("Testing dataset emotion and its count")
test.emotion.value_counts()

#%%#%%Data pre-processing

#convert flatten data to 48*48 matrix
def reshapeTo48and48(dataset):
    #extract pixels value from original pandas dataframe
    pixels_values = dataset.pixels.str.split(" ").tolist()
    #convert pixels of each image to 48*48 formats
    images = []
    for image in np.array(pixels_values, dtype=float):
        images.append(image.reshape(48, 48))
    return np.array(images, dtype=float)

train_images = reshapeTo48and48(train)
test_images = reshapeTo48and48(test)

#reshape to [# of samples][width][height][pixels] for tensorflow-keras input format
train_images = train_images.reshape(train_images.shape[0], 48, 48, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 48, 48, 1).astype('float32')

#check input format
train_images.shape

#normilize the data
train_images = train_images/255
test_images = test_images/255

#One hot encode outputs: change target(emotion) values to input format with one-hot encode
train_targets = np_utils.to_categorical(train.emotion.values)
test_targets = np_utils.to_categorical(test.emotion.values)

#set number of prediction classes
num_classes = test_targets.shape[1]

#%% Show emotion picture from data randomly

def Plot_SomeEmotion_Sample_Randomly(emotion, dataset):
    #select certain emotion sub dataset
    EmotionSet = dataset[dataset.emotion == emotion]
    #randomly select one sample
    pixels_list = EmotionSet.sample(1).pixels.str.split(" ").tolist()
    #convert to 48*48 format
    show_image = np.array(pixels_list, dtype=float).reshape(48,48)
    #plot the image
    plt.imshow(show_image, cmap='gray')
    
#%%
#plot randomly disgust image
Plot_SomeEmotion_Sample_Randomly(1, train)
#%%
#plot randomly happy image
Plot_SomeEmotion_Sample_Randomly(3, train)

#%% CNN MODEL

def build_model(input_shape, num_classes):
    
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
            padding='same', activation='relu', \
            input_shape=input_shape))
    # 48*48*64
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 24*24*64

    model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 24*24*128
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 12*12*128

    model.add(Conv2D(filters=256, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 12*12*256
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 7*7*256

    model.add(Flatten())
    model.add(Dense(1792, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

#%% Training model

#PARAMETERS#
input_shape = (48, 48, 1)
num_classes = 7
MODEL_NAME = 'emotion_recogniser_04'

#set information to save model and for early stopping
filename = "model_04.hdf5"
early_stopping = EarlyStopping(monitor='val_acc', patience=10)
checkpoint = ModelCheckpoint(filename, monitor='val_acc', verbose=2, save_best_only=True, mode='auto')


##complie
model = build_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

    
##begin to train and save training history
history = model.fit(train_images, train_targets, validation_data=(test_images, test_targets), 
                    epochs=100, batch_size=160, callbacks=[early_stopping, checkpoint], verbose=2)

#%% Plot

# plot history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy-Iteration Graph')
plt.ylabel('Accuracy')
plt.xlabel('Iteration(epoch)')
plt.legend(['train',  'test'], loc='upper left')
plt.show()

#%% Load saved trained model & show the final analysis

#build a saved_model and load weights
Saved_model = build_model(input_shape, num_classes)
Saved_model.load_weights("model_04.hdf5")
Saved_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#get loss value and accuracy for testing set
scores = Saved_model.evaluate(test_images, test_targets, verbose=0)

#print accuracy
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#%% Draw confusion matrix to evaluate the final model

Saved_prediction = Saved_model.predict_classes(test_images, verbose=0)
True_prediction = test.emotion.values

cm = confusion_matrix(True_prediction, Saved_prediction)


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, int(cm[i, j]*100)/100.0,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
plot_confusion_matrix(cm, classes=class_names, normalize=False,
                      title='Confusion Matrix for Test Dataset')
plt.show()
#%%

def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")
#%%    
    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")